In [1]:
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline
import time

In [2]:
#to keep things in order, and to avoid to copy and paste everytime our functions if we want to use them in more than one folder,
#we can temporarily use this library. 
import sys

#in this way Python will search the implementations also in the path '../HelperFunctions'
sys.path.insert(0, '../HelperFunctions')
sys.path.insert(0, '../pre-processing/Clean_Data/')

In [3]:
from proj1_helpers import *
from common_functions import *
from counters import *
from remove import *
from replace import *
from regressors import batch_iter

In [4]:
yb, input_data, ids = load_csv_data("../data/train.csv", sub_sample=False)

In [5]:
#this will surely be deleted, in this way we are sure that original_data is the original version of the data and we don't have
#to load them again
from copy import deepcopy
originalData = deepcopy(input_data)
originalY = deepcopy(yb)
print(yb)

idx_wrong=np.where(yb==-1)
print(idx_wrong)
yb[idx_wrong]=0
print(yb)

[ 1. -1. -1. ...  1. -1. -1.]
(array([     1,      2,      3, ..., 249996, 249998, 249999]),)
[1. 0. 0. ... 1. 0. 0.]


# Functions from lab 5

In [38]:
def sig(t):
    """apply sigmoid function on t."""
    return np.exp(t)/(1 + np.exp(t))


#sigmoid = np.vectorize(sig)

def sigmoid(t):
     return .5*(1+np.tanh(.5 * t))

#Is it okay the multiplication between vectors??
# Why do you divide by the number of samples? (mean)
#def calculate_loss(y, tx, w):
#   """compute the cost by negative log likelihood."""
 #   pred = sigmoid(tx.dot(w))
 #   correctZero = 1e-7
   # loss = np.sum((1 - y) * np.log(1 - pred + correctZero) + y * np.log(pred + correctZero))
    #return np.squeeze(- loss)
    #return -loss/y.shape[0]

# Just to try again the function of the professor --> good idea to add a correction to avoid the log(0)
def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood."""
    #pred = sig(tx.dot(w))
    correctZero = 1e-7
    #loss = y.T.dot(np.log(pred+correctZero)) + (1 - y).T.dot(np.log(1 - pred + correctZero))
    #loss= y.T.dot(np.log(pred)) + (1 - y).T.dot(np.log(1 - pred))
    loss = np.ones(tx.shape[0]).dot(np.log(1+np.exp(tx.dot(w))))-y.dot(tx.dot(w))
    return np.squeeze(-loss)

# same question as above
def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    pred = sig(tx.dot(w))
    print('pred =',pred)
    grad = tx.T.dot(pred - y)
    return grad
    #return grad/y.shape[0]

def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descen using logistic regression.
    Return the loss and the updated w.
    """
    #batch = next(batch_iter(y, tx, 32))
    #minibatch_y, minibatch_tx = batch[0], batch[1]
    grad = calculate_gradient(y, tx, w)
    print('grad =', grad)
    loss = calculate_loss(y, tx, w)
    print('loss =', loss)
    w = w - gamma * grad
    return loss, w

In [42]:
#Creation of tx

input_data = deepcopy(originalData)
print(input_data.shape)

#Clean the dataset
#numInvalidValues = countInvalid(input_data,-999)
#idxCols = np.where(numInvalidValues>0)[0]
#input_data = replaceWithZero(input_data,-999,idxCols)

# Stocking the indexes of columns to remove
idxCols = np.where(countInvalid(input_data,-999)>0)[0]
input_data=removeColumns(input_data,0)
print(input_data.shape)

#standardize
tx,_,_ = standardize(input_data)

#add ones
tx = np.c_[np.ones((yb.shape[0], 1)), tx]
y = yb
print(tx.shape)

(250000, 30)
(250000, 19)
(250000, 20)


In [56]:
#To understand better standardization
A=[[1.3,4.7,8],[0.4,5,6],[2,5,1.2]]
print(A)
print(np.mean(A,axis=0))
print(A-np.mean(A,axis=0))
a=A-np.mean(A,axis=0)
b=np.std(A,axis=0)
print('std =',b)
print(a/np.std(A,axis=0))

[[1.3, 4.7, 8], [0.4, 5, 6], [2, 5, 1.2]]
[1.23333333 4.9        5.06666667]
[[ 0.06666667 -0.2         2.93333333]
 [-0.83333333  0.1         0.93333333]
 [ 0.76666667  0.1        -3.86666667]]
std = [0.65489609 0.14142136 2.85345794]
[[ 0.10179732 -1.41421356  1.02799249]
 [-1.2724665   0.70710678  0.32708852]
 [ 1.17066918  0.70710678 -1.35508101]]


In [59]:
#first tries with gradient descent

#Y, TX = sample_data(y, tx, 10, 150000)
max_iter = 100
w = (0.3)*np.ones(tx.shape[1])
gamma = 0.001
loss0 = 0

for iter in range(max_iter):
    w_old = w
    loss, w = learning_by_gradient_descent(y, tx, w, gamma)
    print('w =',w)
    #gamma=0.55*gamma
    if iter<10:
        w_new = w
        print('iter = ', iter)
        #print(f'Iter = {iter}, Loss = {loss}, |w_old - w_new| = {np.linalg.norm(w_old-w_new)}')




pred = [0.86261365 0.5408875  0.94381645 ... 0.23196916 0.12924795 0.29528372]
grad = [ 44950.00262175  24621.69674809  29216.41193362 -16095.05306724
  32408.05372318  40188.62868644  41381.04043039   3983.82038734
  -2799.24031951  18241.54694441   9563.62080475  42236.60148554
  18560.65791435   7959.31203243  33193.56432465  11281.25462266
  40024.50367812  38477.84207328  38574.03289732]
loss = -251852.47360791118
w = [-44.65000262 -24.32169675 -28.91641193  16.39505307 -32.10805372
 -39.88862869 -41.08104043  -3.68382039   3.09924032 -17.94154694
  -9.2636208  -41.93660149 -18.26065791  -7.65931203 -32.89356432
 -10.98125462 -39.72450368 -38.17784207 -38.2740329 ]
iter =  0
pred = [1.09130120e-060 1.00000000e+000 2.68232031e-165 ... 1.00000000e+000
 1.00000000e+000 1.00000000e+000]
grad = [ 28703.76683368 -14638.34817753 -95107.6220295   37617.71306059
 -38488.96562231 -98235.60104415  -9976.24622094 -71519.71233206
 -52392.51832738 -11902.82100603  -3538.31355604 -50245.0795188


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


grad = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
loss = nan
w = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
iter =  5
pred = [nan nan nan ... nan nan nan]
grad = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
loss = nan
w = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
iter =  6
pred = [nan nan nan ... nan nan nan]
grad = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
loss = nan
w = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
iter =  7
pred = [nan nan nan ... nan nan nan]
grad = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
loss = nan
w = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
iter =  8
pred = [nan nan nan ... nan nan nan]
grad = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
loss =

pred = [nan nan nan ... nan nan nan]
grad = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
loss = nan
w = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
pred = [nan nan nan ... nan nan nan]
grad = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
loss = nan
w = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
pred = [nan nan nan ... nan nan nan]
grad = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
loss = nan
w = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
pred = [nan nan nan ... nan nan nan]
grad = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
loss = nan
w = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
pred = [nan nan nan ... nan nan nan]
grad = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
loss = na

loss = nan
w = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
pred = [nan nan nan ... nan nan nan]
grad = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
loss = nan
w = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
pred = [nan nan nan ... nan nan nan]
grad = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
loss = nan
w = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
pred = [nan nan nan ... nan nan nan]
grad = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
loss = nan
w = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
pred = [nan nan nan ... nan nan nan]
grad = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
loss = nan
w = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]
pred = [nan nan nan ... nan nan nan]
g

Let's try to generate a submission

In [ ]:
WWW = w 

In [ ]:
xxx, test_data, ids = load_csv_data("../data/test.csv", sub_sample=False)

In [ ]:
#we need to preprocess the test_data as we preprocessed the train data

numInvalidValues = countInvalid(test_data,-999)
idxCols = np.where(numInvalidValues>0)[0]
test_data = replaceWithZero(test_data,-999,idxCols)
txTest,_,_ = standardize(test_data)
txTest = np.c_[np.ones((xxx.shape[0], 1)), txTest]

In [ ]:
y_pred = predict_labels(WWW, txTest)

In [ ]:
create_csv_submission(ids, y_pred, 'submission.csv')

# Same thing with PCA

In [ ]:
sys.path.insert(0, '../pre-processing/PCA/')

In [ ]:
from pca_functions import PCAWithCovariance

input_data = deepcopy(originalData)
y = deepcopy(originalY)
print(input_data.shape)
print(y.shape)
numInvalidValues = countInvalid(input_data,-999)
idxCols = np.where(numInvalidValues>0)[0]
input_data = replaceWithZero(input_data,-999,idxCols)
input_data,_,_ = standardize(input_data)

In [ ]:
from pca_functions import PCAWithCovariance

input_data = deepcopy(originalData)
y = deepcopy(originalY)
print(input_data.shape)
print(y.shape)
input_data=removeColumns(input_data,0)
print(input_data.shape)
input_data,_,_ = standardize(input_data)

In [ ]:
#Generating the principal components

_,eV = PCAWithCovariance(input_data)

N = 7 #num p. components
components = np.empty(input_data.shape[0])
for i in range(N):
    components = np.c_[components, input_data.dot(eV[:,i])]
    
print(components.shape)
print(components[:,0])
tx = np.c_[np.ones(input_data.shape[0]), components]
print(tx.shape)
print(tx[:,0])
print(tx[:,1])

In [ ]:
Y, TX = sample_data(y, tx, 1, 70000)
TX.shape

In [ ]:
max_iter = 1000
w = np.ones(tx.shape[1])
gamma = 10000
loss0 = 0
for iter in range(max_iter):
    wold = w
    loss, w = learning_by_gradient_descent(yb, tx, w, gamma)
    #gamma=0.55*gamma
    if iter%10==0:
        wnew = w
        print(f'Iter = {iter}, Loss = {loss}, |wold - wnew| = {np.linalg.norm(wold-wnew)}')
print('final loss =',loss)

In [ ]:
# generate submission with PCA
xxx, test_data, ids = load_csv_data("../data/test.csv", sub_sample=False)



In [ ]:
numInvalidValues = countInvalid(test_data,-999)
idxCols = np.where(numInvalidValues>0)[0]
test_data = replaceWithZero(test_data,-999,idxCols)
test_data,_,_ = standardize(test_data)

components = np.empty(test_data.shape[0])
for i in range(N):
    components = np.c_[components, test_data.dot(eV[:,i])]
    
#tx_test = np.c_[np.ones(test_data.shape[0]), components]
tx_test = components

y_pred = predict_labels(w, tx_test)
create_csv_submission(ids, y_pred, 'submission.csv')